In [29]:
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
!pip install pyECLAT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
!pip install pyfpgrowth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
# Import headers
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [69]:
#Data preprocessing
dataset = pd.read_csv('/content/groceries - groceries.csv', header = 0)
Data=dataset
rows = len(Data.axes[0])
cols = len(Data.axes[1])
Data.columns = range(Data.shape[1])
rows

9834

In [70]:
# Intializing the list
transacts = []
# populating a list of transactions
for i in range(0, rows): 
  transacts.append([str(Data.values[i,j]) for j in range(0, cols)])


In [71]:
#Apriori algorithm
from apyori import apriori
ap = apriori(transactions = transacts, min_support = 0.2, use_colnames=True) # generate rules
# list of results coming from the apriori model
rslt = list(ap)

rslt

[RelationRecord(items=frozenset({'nan'}), support=0.9998983119788489, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'nan'}), confidence=0.9998983119788489, lift=1.0)]),
 RelationRecord(items=frozenset({'whole milk'}), support=0.2555419971527354, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'whole milk'}), confidence=0.2555419971527354, lift=1.0)]),
 RelationRecord(items=frozenset({'whole milk', 'nan'}), support=0.2554403091315843, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'whole milk', 'nan'}), confidence=0.2554403091315843, lift=1.0), OrderedStatistic(items_base=frozenset({'nan'}), items_add=frozenset({'whole milk'}), confidence=0.2554662869927794, lift=0.9997037271337018), OrderedStatistic(items_base=frozenset({'whole milk'}), items_add=frozenset({'nan'}), confidence=0.9996020692399522, lift=0.9997037271337018)])]

In [72]:
%timeit -n 100 -r 10 ap

100 loops, best of 10: 61.1 ns per loop


In [73]:
#ECLAT algorithm
from pyECLAT import ECLAT

# loading transactions DataFrame to ECLAT class
eclat = ECLAT(data=Data)

# DataFrame of binary values
eclat.df_bin


,baby cosmetics,flour,turkey,hard cheese,brandy,misc. beverages,domestic eggs,salt,organic products,syrup,...,soft cheese,brown bread,make up remover,kitchen towels,canned vegetables,herbs,cream,butter,specialty chocolate,tidbits
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9829,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9830,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9831,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9832,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
items_total = eclat.df_bin.astype(int).sum(axis=0)

items_total

baby cosmetics           6
flour                  171
turkey                  80
hard cheese            241
brandy                  41
                      ... 
herbs                  160
cream                   13
butter                 545
specialty chocolate    299
tidbits                 23
Length: 169, dtype: int64

In [75]:
# count items in each row
items_per_transaction = eclat.df_bin.astype(int).sum(axis=1)

items_per_transaction

0        3
1        1
2        4
3        4
4        5
        ..
9829    17
9830     1
9831    10
9832     4
9833     5
Length: 9834, dtype: int64

In [76]:
df = pd.DataFrame({'items': items_total.index, 'transactions': items_total.values}) 

# cloning pandas DataFrame for visualization purpose  
df_table = df.sort_values("transactions", ascending=False)

#  Top 5 most popular products/items
df_table.head(5)

,items,transactions
15,whole milk,2513
72,other vegetables,1903
54,rolls/buns,1809
70,soda,1715
107,yogurt,1372


In [89]:
eclat_indices, eclat_supports = eclat.fit(min_support=0.2)

Combination 1 by 1


1it [00:00, 32.53it/s]


Combination 2 by 2


0it [00:00, ?it/s]


Combination 3 by 3


0it [00:00, ?it/s]


In [90]:
print(eclat_supports)

{'whole milk': 0.2555419971527354}


In [97]:
%timeit -n 100 -r 10 eclat_supports

100 loops, best of 10: 53 ns per loop


In [80]:
import pyfpgrowth

In [81]:
min_support= 0.2*rows
patterns = pyfpgrowth.find_frequent_patterns(transacts, min_support)
patterns

{('nan',): 271325,
 ('nan', 'nan'): 3670683,
 ('nan', 'whole milk'): 63422,
 ('whole milk',): 2513}

In [99]:
%timeit -n 100 -r 10 patterns

100 loops, best of 10: 50.2 ns per loop
